In [5]:
import os
import json
import re
import pandas as pd
import requests
from dotenv import load_dotenv

# === Загрузка переменных окружения ===
load_dotenv()

# === Конфигурация токена и URL Gemini API ===
GEMINI_API_KEY = os.getenv("GEM_API_TOKEN")
GEMINI_API_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"
HEADERS = {"Content-Type": "application/json"}

# === Вызов Gemini API ===
def call_gemini(prompt: str) -> str:
    response = requests.post(
        f"{GEMINI_API_URL}?key={GEMINI_API_KEY}",
        headers=HEADERS,
        json={"contents": [{"parts": [{"text": prompt}]}]},
        timeout=30,
    )
    response.raise_for_status()
    return response.json()["candidates"][0]["content"]["parts"][0]["text"]

# === Очистка и парсинг ответа Gemini ===
def clean_response(raw: str) -> dict:
    cleaned = re.sub(r"^```(?:json)?\\n?|```$", "", raw.strip(), flags=re.IGNORECASE | re.MULTILINE).strip()
    cleaned = cleaned.replace('\u200b', '').replace('\ufeff', '')
    return json.loads(cleaned)

# === Шаблоны промптов ===
SUMMARY_PROMPT_TEMPLATE = """
Разбей текст описания вакансии на три кратких блока:

1. 📌 *О компании* — в 1–2 предложениях.
2. 🧾 *Обязанности* — только ключевые пункты, кратко, по делу (до 3–5 пунктов).
3. 🎯 *Требования* — самые важные навыки и условия, кратко (до 3–5 пунктов).

📢 Не пиши вводных фраз, не добавляй лишние слова. Используй маркированный список (•), если возможно.
🚫 Если для какого-либо блока нет информации — укажи "Не указано".
🔍 Не придумывай ничего нового — работай только с тем, что есть в описании.

Верни чистый JSON:
```json
{{
  "about_company": "...",
  "responsibilities": "...",
  "requirements": "..."
}}
"""

FILTER_PROMPT = """
Ты ассистент, который определяет релевантность вакансии по названию и описанию.

Профессия считается релевантной, если она относится к одной из следующих:
- Data Scientist
- Senior Data Scientist
- Junior Data Scientist
- Machine Learning Engineer
- ML Engineer
- Data Analyst
- Senior Data Analyst
- Data Engineer
- Big Data Engineer
- Data Architect
- Business Intelligence Analyst
- BI Analyst
- Business Intelligence Developer
- Statistician
- Quantitative Analyst
- NLP Engineer
- Computer Vision Engineer
- Deep Learning Engineer
- Artificial Intelligence Engineer
- AI Researcher
- Data Researcher
- Predictive Analytics Specialist
- Data Science Manager
- Analytics Consultant
- Data Miner
- Data Specialist
- Data Modeler

Профессия: "{title}"
Описание: "{description}"

Ответь строго одним словом: yes или no.
"""

# === Генерация summary ===
def generate_summary(description: str) -> dict:
    if not description or len(description.strip()) < 50:
        return {"about_company": "Не указано", "responsibilities": "Не указано", "requirements": "Не указано"}

    prompt = SUMMARY_PROMPT_TEMPLATE.format(description=description)
    try:
        raw = call_gemini(prompt)
        return clean_response(raw)
    except Exception as e:
        print(f"❌ Ошибка генерации summary: {e}")
        return {"about_company": "Не указано", "responsibilities": "Не указано", "requirements": "Не указано"}

# === Фильтрация вакансий ===
def filter_vacancy(title: str, description: str) -> bool:
    prompt = FILTER_PROMPT.format(title=title, description=description)
    try:
        raw = call_gemini(prompt)
        if not raw.strip():
            print(f"❌ Пустой ответ от Gemini при генерации summary.")
            return {"about_company": "Не указано", "responsibilities": "Не указано", "requirements": "Не указано"}
        return clean_response(raw)
    except Exception as e:
        print(f"❌ Ошибка генерации summary: {e}")
        return {"about_company": "Не указано", "responsibilities": "Не указано", "requirements": "Не указано"}




In [13]:
import os
import json
import re
import pandas as pd
import requests
from dotenv import load_dotenv

# === Загрузка переменных окружения ===
load_dotenv()

# === Конфигурация токена и URL Gemini API ===
GEMINI_API_KEY = os.getenv("GEM_API_TOKEN")
GEMINI_API_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"
HEADERS = {"Content-Type": "application/json"}

# === Вызов Gemini API ===
def call_gemini(prompt: str) -> str:
    response = requests.post(
        f"{GEMINI_API_URL}?key={GEMINI_API_KEY}",
        headers=HEADERS,
        json={"contents": [{"parts": [{"text": prompt}]}]},
        timeout=30,
    )
    response.raise_for_status()
    return response.json()["candidates"][0]["content"]["parts"][0]["text"]

# === Безопасный парсинг JSON ответа Gemini ===
def safe_parse_json(raw: str) -> dict:
    cleaned = re.sub(r"^```(?:json)?\n?|```$", "", raw.strip(), flags=re.IGNORECASE | re.MULTILINE).strip()
    cleaned = cleaned.replace('\u200b', '').replace('\ufeff', '')
    cleaned = cleaned.replace('\\n', ' ').replace('\\', '')

    if not (cleaned.startswith('{') and cleaned.endswith('}')):
        print(f"❌ Невалидный JSON-ответ от Gemini (не начинается и не заканчивается на {{}}):\n{cleaned}")
        return {"about_company": "Не указано", "responsibilities": "Не указано", "requirements": "Не указано"}

    try:
        parsed = json.loads(cleaned)

        if not isinstance(parsed, dict):
            print(f"❌ Ожидался объект dict, но пришло: {type(parsed)} → {parsed}")
            return {"about_company": "Не указано", "responsibilities": "Не указано", "requirements": "Не указано"}

        def process_field(value):
            if isinstance(value, list):
                return " ".join(item.strip() for item in value if isinstance(item, str))
            return str(value).strip()

        about_company = process_field(parsed.get("about_company", "Не указано"))
        responsibilities = process_field(parsed.get("responsibilities", "Не указано"))
        requirements = process_field(parsed.get("requirements", "Не указано"))

        return {
            "about_company": about_company if about_company else "Не указано",
            "responsibilities": responsibilities if responsibilities else "Не указано",
            "requirements": requirements if requirements else "Не указано",
        }

    except Exception as e:
        print(f"❌ Ошибка парсинга JSON: {e}\nОтвет от Gemini:\n{cleaned}")
        return {"about_company": "Не указано", "responsibilities": "Не указано", "requirements": "Не указано"}

# === Шаблоны промптов ===
SUMMARY_PROMPT_TEMPLATE = """
Разбей текст описания вакансии на три кратких блока:

1. 📌 *О компании* — в 1–2 предложениях.
2. 🧾 *Обязанности* — только ключевые пункты, кратко, по делу (до 3–5 пунктов).
3. 🎯 *Требования* — самые важные навыки и условия, кратко (до 3–5 пунктов).

📢 Не пиши вводных фраз, не добавляй лишние слова. Используй маркированный список (•), если возможно.
🚫 Если для какого-либо блока нет информации — укажи "Не указано".
🔍 Не придумывай ничего нового — работай только с тем, что есть в описании.

Верни чистый JSON:
```json
{{
  "about_company": "...",
  "responsibilities": "...",
  "requirements": "..."
}}
```

Описание вакансии:
{description}
"""

FILTER_PROMPT = """
Ты ассистент, который определяет релевантность вакансии по названию и описанию.

Профессия считается релевантной, если она относится к одной из следующих:
- Data Scientist
- Senior Data Scientist
- Junior Data Scientist
- Machine Learning Engineer
- ML Engineer
- Data Analyst
- Senior Data Analyst
- Data Engineer
- Big Data Engineer
- Data Architect
- Business Intelligence Analyst
- BI Analyst
- Business Intelligence Developer
- Statistician
- Quantitative Analyst
- NLP Engineer
- Computer Vision Engineer
- Deep Learning Engineer
- Artificial Intelligence Engineer
- AI Researcher
- Data Researcher
- Predictive Analytics Specialist
- Data Science Manager
- Analytics Consultant
- Data Miner
- Data Specialist
- Data Modeler

Профессия: "{title}"
Описание: "{description}"

Ответь строго одним словом: yes или no.
"""

# === Генерация summary ===
def generate_summary(description: str) -> dict:
    if not description or len(description.strip()) < 50:
        return {"about_company": "Не указано", "responsibilities": "Не указано", "requirements": "Не указано"}

    prompt = SUMMARY_PROMPT_TEMPLATE.format(description=description)
    try:
        raw = call_gemini(prompt)
        if not raw.strip():
            print(f"❌ Пустой ответ от Gemini при генерации summary.")
            return {"about_company": "Не указано", "responsibilities": "Не указано", "requirements": "Не указано"}
        return safe_parse_json(raw)
    except Exception as e:
        print(f"❌ Ошибка генерации summary: {e}")
        return {"about_company": "Не указано", "responsibilities": "Не указано", "requirements": "Не указано"}

# === Фильтрация вакансий ===
def filter_vacancy(title: str, description: str) -> bool:
    prompt = FILTER_PROMPT.format(title=title, description=description)
    try:
        raw = call_gemini(prompt)
        if not raw.strip():
            print(f"❌ Пустой ответ от Gemini при фильтрации вакансии.")
            return False
        return raw.strip().lower() == "yes"
    except Exception as e:
        print(f"❌ Ошибка фильтрации вакансии: {e}")
        return False

# === Загрузка своего DataFrame ===
df = pd.read_csv('C:/Users/User/hh_data_project/data/processed/vacancies_clean_2025-04-26.csv')

# Удаление NaN значений в важных колонках
df = df.dropna(subset=['title', 'description'])

# === Реальный тестовый прогон ===
results = []

for idx, row in df.iterrows():
    title = row.get('title', '')
    description = row.get('description', '')

    is_relevant = filter_vacancy(title, description)
    print(f"📋 {title} → {'✅ Релевантно' if is_relevant else '❌ Нерелевантно'}")

    if is_relevant:
        summary = generate_summary(description)
        results.append({
            'title': title,
            'summary': summary
        })

print(f"✅ Найдено {len(results)} релевантных вакансий.")

📋 Data Engineer (Middle+) → ✅ Релевантно
📋 Data Scientist / ML Engineer → ✅ Релевантно
❌ Ошибка парсинга JSON: Expecting ',' delimiter: line 2 column 25 (char 26)
Ответ от Gemini:
{
  "about_company": "АО "Банковское Сервисное Бюро Национального банка" - ведущая финансовая организация, предоставляющая сервисные решения для Национального Банка Казахстана. Развивает направления цифровой трансформации и обработки данных.",
  "responsibilities": "• Разработка и внедрение моделей машинного обучения. • Анализ и подготовка данных. • Построение и тестирование моделей, подбор гиперпараметров. • Внедрение моделей в продуктивные системы и сопровождение. • Использование инструментов для разработки ML-решений (Python, Jupyter, scikit-learn, TensorFlow, PyTorch и др.).",
  "requirements": "• Опыт разработки и внедрения моделей машинного обучения и анализа данных. • Глубокие знания Python и библиотек для Data Science. • Опыт работы с фреймворками глубокого обучения (TensorFlow, PyTorch) - преимуществ

In [3]:
import pandas as pd
df = pd.read_csv("C:/Users/User/hh_data_project/data/main.csv")
df['work_format'].unique()

array(['Формат работы: на месте работодателя', 'Формат работы: удалённо',
       'Формат работы: на месте работодателя, удалённо или разъездной',
       'Формат работы: гибрид',
       'Формат работы: на месте работодателя или гибрид', 'Не указано',
       'Формат работы: удалённо или разъездной',
       'Формат работы: удалённо или гибрид',
       'Формат работы: на месте работодателя или удалённо',
       'Формат работы: на месте работодателя, удалённо или гибрид',
       'Формат работы: на месте работодателя или разъездной'],
      dtype=object)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("C:/Users/User/hh_data_project/data/processed/vacancies_clean_2025-05-05.csv")

In [3]:
df

,title,company,location,salary,description,experience,employment_type,schedule,working_hours,work_format,published_date,link,skills,salary_range,published_date_dt
0,Data Engineer,АО Страховая компания Казахмыс,Алматы,"до 700 000 ₸ за месяц, на руки","Обязанности:\r\n\r\nПроектирование, разработка...",1–3 года,Полная занятость,График: 5/2,8,На месте работодателя,5 мая 2025,https://hh.kz/vacancy/120225611?query=Data+Eng...,Не указано,до 700000 ₸,2025-05-05
1,Data Engineer,АО Страховая компания Казахмыс,Алматы,"до 700 000 ₸ за месяц, на руки","Обязанности:\r\n\r\nПроектирование, разработка...",1–3 года,Полная занятость,График: 5/2,8,На месте работодателя,5 мая 2025,https://hh.kz/vacancy/120225611?query=ML+Engin...,Не указано,до 700000 ₸,2025-05-05
2,Data Engineer,АО Страховая компания Казахмыс,Алматы,"до 700 000 ₸ за месяц, на руки","Обязанности:\r\n\r\nПроектирование, разработка...",1–3 года,Полная занятость,График: 5/2,8,На месте работодателя,5 мая 2025,https://hh.kz/vacancy/120225611?query=BI+Analy...,Не указано,до 700000 ₸,2025-05-05
3,Junior Data Engineer,АО ForteBank,Не указано,"от 150 000 ₸ за месяц, на руки","ВАКАНСИЯ ТАКЖЕ ДЛЯ УЧЕНИКОВ ПОСЛЕДНЕГО КУРСА, ...",не требуется,Частичная занятость\r\nВозможно временное офор...,График: 5/2,8,На месте работодателя,5 мая 2025,https://hh.kz/vacancy/120202348?query=Data+Spe...,Не указано,от 150000 ₸,2025-05-05
4,Senior QA Automation Engineer (Java),Иностр. п. ОбъектСтиль/ Object Style,Не указано,Не указано,ObjectStyle is a provider of open source solut...,более 6 лет,Полная занятость\r\nВозможно временное оформление,График: 5/2,8,Удалённо,5 мая 2025,https://hh.kz/vacancy/120231161?query=Data+Eng...,Не указано,Не указано,2025-05-05
5,RRP Product Manager,ТОО JTI Kazakhstan,Не указано,Не указано,RRP Product Manager\r\n\r\nThe Product Manager...,не требуется,Полная занятость,График: 5/2,8,Гибрид,5 мая 2025,https://hh.kz/vacancy/120214879?query=Big+Data...,Не указано,Не указано,2025-05-05
6,Strategic Product Analyst,ТОО G5EN KAZ,Не указано,Не указано,G5 Gamesis a Swedish developer and publisher o...,1–3 года,Полная занятость,График: 5/2,8,Удалённо,5 мая 2025,https://hh.kz/vacancy/120221567?query=Data+Res...,Не указано,Не указано,2025-05-05
7,Digital Solutions CC Senior Specialist,ТОО Филип Моррис Казахстан,Не указано,Не указано,Be a part of a revolutionary change\r\n\r\nAt ...,1–3 года,Полная занятость,График: 5/2,8,Гибрид,5 мая 2025,https://hh.kz/vacancy/120198515?query=Data+Res...,Не указано,Не указано,2025-05-05
8,Бизнес-аналитик,АО КСЖ Standard Life,Алматы,Не указано,Обязанности:\r\n\r\nСбор и анализ требований:\...,1–3 года,Полная занятость,График: 5/2,8,Не указано,5 мая 2025,https://hh.kz/vacancy/120228009?query=BI+Analy...,Не указано,Не указано,2025-05-05
9,Data & Infrastructure Engineer,Шлюмберже Лоджелко Инк. в РК,Не указано,Не указано,Responsibilities:\r\n\r\nManaging cloud enviro...,3–6 лет,Полная занятость,График: 5/2,8,На месте работодателя,5 мая 2025,https://hh.kz/vacancy/120192223?query=Data+Arc...,Не указано,Не указано,2025-05-05
